Quelle: https://colab.research.google.com/github/deepset-ai/haystack/blob/main/tutorials/Tutorial5_Evaluation.ipynb

# Bewertung einer Pipeline und ihrer Komponenten

Um eine Aussage über die Qualität der Ergebnisse machen zu können, die eine Frage-Antwort-Pipeline oder eine andere Pipeline in haystack produziert, ist es wichtig, sie zu evaluieren. Darüber hinaus lässt sich durch die Bewertung feststellen, welche Komponenten der Pipeline verbessert werden können.


In [3]:
doc_index = "tutorial5_docs"
label_index = "tutorial5_labels"

In [4]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
    host="localhost",
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

In [ ]:
from haystack.nodes import PreProcessor


preprocessor = PreProcessor(
    split_by="word",
    split_length=1000,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)

# Die Methode add_eval_data() konvertiert den angegebenen Datensatz im json-Format in Haystack-Dokument- und Label-Objekte. Diese Objekte werden dann in ihrem jeweiligen Dokument- und Label-Index im Dokumentenspeicher indiziert.
document_store.add_eval_data(
    filename="QA/Evaluation/Datensatz_Evaluation/Datensatz_Evaluation.json",
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

## Initialisierung Retriever und Reader

In [ ]:
# Initialize Retriever
from haystack.nodes import ElasticsearchRetriever

retriever = ElasticsearchRetriever(document_store=document_store)

In [7]:
# Initialize Reader
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="QA/Finetuning/my_model", top_k=3, return_no_answer=True)

from haystack.pipelines import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

## Evaluation ExtractiveQAPipeline
Das Dokument wird als richtig gefunden eingestuft, wenn es den goldenen Antwortstring enthält.


In [8]:
from haystack.schema import EvaluationResult, MultiLabel


eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)

eval_result = pipeline.eval(labels=eval_labels, params={"Retriever": {"top_k": 5}})

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  7.29 Batches/s]


In [9]:
retriever_result = eval_result["Retriever"]
retriever_result.head()

,multilabel_id,query,filters,context,gold_contexts,gold_id_match,context_match,answer_match,gold_id_or_answer_match,gold_id_and_answer_match,...,rank,document_id,gold_document_ids,gold_documents_id_match,gold_contexts_similarity,gold_answers_match,type,node,eval_mode,index
0,b37aacc22d27b4eb0e948a927e713699,Sind Praktika Pflicht?,b'null',"5Sofern nichts anderes bestimmt ist, ergibt sich die Zusammensetzung der \nF...",[Betriebswirtschaftslehre B.Sc. - Universität Regensburg\nZu Hauptinhalt spr...,0.0,0.0,0.0,0.0,0.0,...,1.0,3ffed4883285b0d82b9f72f580f19721-1,[ea5dab07c1d988109eef8590dd9c819e-0],[0.0],[43.68995633187773],[0.0],document,Retriever,integrated,0
1,b37aacc22d27b4eb0e948a927e713699,Sind Praktika Pflicht?,b'null',Nach Abschluss des Moduls sind die Studierenden in \nerwerbende Kompetenzen...,[Betriebswirtschaftslehre B.Sc. - Universität Regensburg\nZu Hauptinhalt spr...,0.0,0.0,0.0,0.0,0.0,...,2.0,1b01fd617010656fbd4dce974747e4b2-3,[ea5dab07c1d988109eef8590dd9c819e-0],[0.0],[41.1353711790393],[0.0],document,Retriever,integrated,1
2,b37aacc22d27b4eb0e948a927e713699,Sind Praktika Pflicht?,b'null',Beratungsprozessen. Kenntnisse in \nqualitativen Forschungsmethoden und dere...,[Betriebswirtschaftslehre B.Sc. - Universität Regensburg\nZu Hauptinhalt spr...,0.0,0.0,0.0,0.0,0.0,...,3.0,3d3bda689fb575437d7eb412334d3e7a-1,[ea5dab07c1d988109eef8590dd9c819e-0],[0.0],[41.288209606986904],[0.0],document,Retriever,integrated,2
3,b37aacc22d27b4eb0e948a927e713699,Sind Praktika Pflicht?,b'null',Informationswissenschaft B.A. - Universität Regensburg\nZu Hauptinhalt sprin...,[Betriebswirtschaftslehre B.Sc. - Universität Regensburg\nZu Hauptinhalt spr...,0.0,1.0,0.0,0.0,0.0,...,4.0,2c2d6e7930a31b4ada8b0c5428a5d3a9-0,[ea5dab07c1d988109eef8590dd9c819e-0],[0.0],[65.74235807860262],[0.0],document,Retriever,integrated,3
4,b37aacc22d27b4eb0e948a927e713699,Sind Praktika Pflicht?,b'null',Amerikanistik (American Studies) B.A. - Universität Regensburg\nZu Hauptinha...,[Betriebswirtschaftslehre B.Sc. - Universität Regensburg\nZu Hauptinhalt spr...,0.0,0.0,0.0,0.0,0.0,...,5.0,532fc76237ed27a6cbe2057fcef7f6da-0,[ea5dab07c1d988109eef8590dd9c819e-0],[0.0],[62.314410480349345],[0.0],document,Retriever,integrated,4


In [10]:
reader_result = eval_result["Reader"]
reader_result.head()

,multilabel_id,query,filters,gold_answers,answer,context,exact_match,f1,exact_match_context_scope,f1_context_scope,...,offsets_in_document,gold_offsets_in_documents,gold_answers_exact_match,gold_answers_f1,gold_documents_id_match,gold_contexts_similarity,type,node,eval_mode,index
0,b37aacc22d27b4eb0e948a927e713699,Sind Praktika Pflicht?,b'null',[Praktika werden empfohlen. Im Wahlmodul kann ein Praktikum angerechnet werden],"Praktika sind nicht verpflichtend, werden aber von vielen Studierenden freiw...","h der neuen Medien.\nPraktika\nPraktika sind nicht verpflichtend, werden abe...",0.0,0.190476,0.0,0.000000,...,"[{'start': 2369, 'end': 2462}]","[{'start': 1524, 'end': 1601}]",[0],[0.1904761904761905],[0.0],[46.0],answer,Reader,integrated,0
1,b37aacc22d27b4eb0e948a927e713699,Sind Praktika Pflicht?,b'null',[Praktika werden empfohlen. Im Wahlmodul kann ein Praktikum angerechnet werden],,None,0.0,0.000000,0.0,0.000000,...,"[{'start': 0, 'end': 0}]","[{'start': 1524, 'end': 1601}]",[0],[0],[0.0],[0.0],answer,Reader,integrated,1
2,b37aacc22d27b4eb0e948a927e713699,Sind Praktika Pflicht?,b'null',[Praktika werden empfohlen. Im Wahlmodul kann ein Praktikum angerechnet werden],"Es wird allen Studierenden empfohlen, neben dem Studium durch geeignete Prak...","raktika\nEs wird allen Studierenden empfohlen, neben dem Studium durch geeig...",0.0,0.142857,0.0,0.000000,...,"[{'start': 3317, 'end': 3451}]","[{'start': 1524, 'end': 1601}]",[0],[0.14285714285714285],[0.0],[43.333333333333336],answer,Reader,integrated,2
0,708f55715e1069068ba018963d2f7ac9,Wie lange ist die Regelstudienzeit?,b'null',"[Die Regelstudienzeit, d.h. die Zeit, in der das vorgesehene Studienprogramm...",6 Semester,"gesehene Studienprogramm idealerweise absolviert werden kann, beträgt 6 Seme...",0.0,0.210526,0.0,0.210526,...,"[{'start': 3746, 'end': 3756}]","[{'start': 1822, 'end': 1953}]",[0],[0.21052631578947367],[0.0],[100.0],answer,Reader,integrated,0
1,708f55715e1069068ba018963d2f7ac9,Wie lange ist die Regelstudienzeit?,b'null',"[Die Regelstudienzeit, d.h. die Zeit, in der das vorgesehene Studienprogramm...",6 Semester,"gesehene Studienprogramm idealerweise absolviert werden kann, beträgt 6 Seme...",0.0,0.210526,0.0,0.210526,...,"[{'start': 4246, 'end': 4256}]","[{'start': 1822, 'end': 1953}]",[0],[0.21052631578947367],[0.0],[100.0],answer,Reader,integrated,1


In [15]:
eval_result.save("../")

## Berechnung von Bewertungskennzahlen

In [16]:
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

Retriever - Recall (single relevant document): 0.6483516483516484
Retriever - Recall (multiple relevant documents): 0.611904761904762
Retriever - Mean Reciprocal Rank: 0.4228937728937729
Retriever - Precision: 0.20109890109890108
Retriever - Mean Average Precision: 0.40167887667887675
Reader - F1-Score: 0.39292586658520695
Reader - Exact Match: 0.0989010989010989


## Erzeugen eines Auswertungsberichts
Eine Zusammenfassung der Bewertungsergebnisse kann ausgedruckt werden, um einen schnellen Überblick zu erhalten. Sie enthält einige aggregierte Metriken und zeigt auch einige falsch vorhergesagte Beispiele.

In [17]:
pipeline.print_eval_report(saved_eval_result)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit: 0.648
                        |
                      Reader
                        |
                        | exact_match: 0.0989
                        | exact_match_top_1: 0.0769
                        | f1: 0.393
                        | f1_top_1: 0.264
                        | num_examples_for_eval:  91.0
                        | num_examples_for_eval_top_1:  91.0
                        |
                      Output

                Wrong Retriever Examples
Query: 
 	Sind Praktika Pflicht?
Gold Document Ids: 
 	ea5dab07c1d988109eef8590dd9c819e-0
Metrics: 
 	recall_multi_hit: 0.0
 	recall_single_hit: 0.0
 	precision: 0.0
 	map: 0.0
 	mrr: 0.0
 	ndcg: 0.0
Documents: 
 	context: 5Sofern nichts anderes be

## Fortgeschrittene Bewertungsmetriken
Als erweiterte Bewertungsmetrik kann die semantic answer similarity (SAS) berechnet werden. Diese Metrik berücksichtigt, ob die Bedeutung einer vorhergesagten Antwort der annotierten Gold-Antwort ähnlich ist, anstatt nur einen String-Vergleich durchzuführen.
Zu diesem Zweck stützt sich SAS auf vortrainierte Modelle.

In [ ]:
advanced_eval_result = pipeline.eval(
    labels=eval_labels, params={"Retriever": {"top_k": 5}}, sas_model_name_or_path="deepset/gbert-large-sts"
)

metrics = advanced_eval_result.calculate_metrics()


In [19]:
print(metrics["Reader"]["sas"])

0.57644486
